In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/train_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/test_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/val_khan_acad.csv" /content


In [4]:
!pip install transformers==3.2.0

     |████████████████████████████████| 1.0MB 4.1MB/s 
     |████████████████████████████████| 890kB 8.3MB/s 
     |████████████████████████████████| 3.0MB 11.9MB/s 
     |████████████████████████████████| 1.2MB 37.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=be268c230462348f3f23aceddcaaa88a200c3c5aafb13ecd30aef363e958b190
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
!pip install git+https://github.com/geoopt/geoopt.git
! pip install git+https://github.com/ferrine/hyrnn.git

  Cloning https://github.com/geoopt/geoopt.git to /tmp/pip-req-build-j9eai644
  Running command git clone -q https://github.com/geoopt/geoopt.git /tmp/pip-req-build-j9eai644
  Created wheel for geoopt: filename=geoopt-0.3.1-cp37-none-any.whl size=76168 sha256=ad685f35d27331890c2c5313fbfee3aee4e8d18e25bd694ab25a661c5c563830
  Stored in directory: /tmp/pip-ephem-wheel-cache-ekfupm_2/wheels/10/df/30/e0d857f034c142ca5f38af048b62aae3da773b272553e5dd21
Successfully built geoopt
  Cloning https://github.com/ferrine/hyrnn.git to /tmp/pip-req-build-pqrvhjtc
  Running command git clone -q https://github.com/ferrine/hyrnn.git /tmp/pip-req-build-pqrvhjtc
  Created wheel for hyrnn: filename=hyrnn-0.0.0-cp37-none-any.whl size=13955 sha256=958c3fa1027c85a721c9303f2b07a09deff16085e41d03bfc4f2348d9a1b3ea9
  Stored in directory: /tmp/pip-ephem-wheel-cache-4sc0oayd/wheels/24/c3/64/cc0e9d25d466081dc154a2a8843157f54d845b916b4ba66418
Successfully built hyrnn


In [6]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.6MB 46kB/s 
     |████████████████████████████████| 368kB 37.9MB/s 
     |████████████████████████████████| 3.2MB 35.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [7]:
import pandas as pd
train_data = pd.read_csv("train_khan_acad.csv")
val_data = pd.read_csv("val_khan_acad.csv")
test_data = pd.read_csv("test_khan_acad.csv")

train_data


,video_transcripts,hierarchy
0,In the last couple of videos we saw that we c...,math>>multivariable-calculus>>multivariable-de...
1,- What we're going to do in this video is gi...,science>>ap-biology>>natural-selection
2,"So once again, we have three equal, or we say...",math>>pre-algebra>>pre-algebra-equations-expre...
3,- Liz's math test included a survey question...,math>>engageny-alg-1>>alg1-2
4,- The following two equations form a linear s...,math>>algebra-home>>alg-system-of-equations
...,...,...
4183,- Hello everyone. So this is what I might ca...,math>>multivariable-calculus>>multivariable-de...
4184,- Let's try now to subtract some two-digit n...,math>>early-math>>cc-early-math-add-sub-100
4185,- Let's say that I have a circle. My best att...,math>>engageny-geo>>geo-5
4186,- So let's look at the female reproductive cy...,science>>health-and-medicine>>human-anatomy-an...


In [8]:
from google.colab import files

In [9]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [10]:
!cp -r "/content/drive/My Drive/Information_retrieval_project/model_hyperIM_khan_acad/" /content/

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Loading BERT tokenizer...


In [12]:
train_data["hierarchy"].value_counts()

science>>health-and-medicine>>circulatory-system-diseases                   99
science>>health-and-medicine>>human-anatomy-and-physiology                  65
science>>health-and-medicine>>respiratory-system-diseases                   55
science>>health-and-medicine>>circulatory-system                            54
science>>health-and-medicine>>infectious-diseases                           52
                                                                            ..
math>>statistics-probability>>advanced-regression-inference-transforming     1
math>>precalculus>>x9e81a4f98389efdf:vectors                                 1
math>>in-in-class-7-math-india-icse>>in-in-7-pair-of-angles-icse             1
math>>algebra-basics>>alg-basics-expressions-with-exponents                  1
math>>4th-grade-foundations-engageny>>4th-m5-engage-ny-foundations           1
Name: hierarchy, Length: 569, dtype: int64

In [13]:
test_data["hierarchy"].value_counts()

science>>health-and-medicine>>human-anatomy-and-physiology               24
science>>health-and-medicine>>circulatory-system-diseases                22
science>>health-and-medicine>>circulatory-system                         17
science>>electrical-engineering>>robots                                  11
math>>algebra-home>>alg-polynomials                                      11
                                                                         ..
math>>in-in-class-7-math-india-icse>>in-in-7-linear-inequalities-icse     1
economics-finance-domain>>macroeconomics>>forex-trade-topic               1
math>>in-in-grade-12-ncert>>in-in-advanced-differentiation-two            1
math>>ap-statistics>>inference-slope-linear-regression                    1
math>>calculus-2>>cs2-integration-techniques                              1
Name: hierarchy, Length: 416, dtype: int64

In [14]:

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
LE.fit_transform(pd.concat([train_data['hierarchy'],test_data['hierarchy']]))

train_data['label'] = LE.transform(train_data['hierarchy'])
train_data.head()

,video_transcripts,hierarchy,label
0,In the last couple of videos we saw that we c...,math>>multivariable-calculus>>multivariable-de...,354
1,- What we're going to do in this video is gi...,science>>ap-biology>>natural-selection,422
2,"So once again, we have three equal, or we say...",math>>pre-algebra>>pre-algebra-equations-expre...,384
3,- Liz's math test included a survey question...,math>>engageny-alg-1>>alg1-2,231
4,- The following two equations form a linear s...,math>>algebra-home>>alg-system-of-equations,99


In [15]:
test_data['label'] = LE.transform(test_data['hierarchy'])
test_data.head()

,video_transcripts,hierarchy,label
0,- What I hope to do in this video is get fam...,math>>math1>>x89d82521517266d4:functions,335
1,In the last video we were able to set up this...,math>>old-ap-calculus-ab>>ab-applications-defi...,357
2,- In previous videos we talk about GDP as th...,economics-finance-domain>>ap-macroeconomics>>e...,3
3,- So what we're gonna do in this video is se...,math>>old-integral-calculus>>definite-integral...,378
4,- So I've said that if you have a vector fie...,math>>multivariable-calculus>>multivariable-de...,354


In [16]:
val_data['label'] = LE.transform(val_data['hierarchy'])
val_data.head()

,video_transcripts,hierarchy,label
0,Find the probability of rolling doubles on tw...,math>>precalculus>>x9e81a4f98389efdf:prob-comb,395
1,"After the food is swallowed, it leaves the m...",science>>health-and-medicine>>human-anatomy-an...,497
2,Let's now talk about what is easily one of th...,math>>geometry>>hs-geo-trig,256
3,The goal in this video is to essentially prov...,science>>chemistry>>thermodynamics-chemistry,472
4,"A line goes through the points (-1, 6) and (5...",math>>in-in-grade-11-ncert>>in-in-class11-stra...,304


In [17]:
def get_labels(prediction):
    predicted_label =  LE.inverse_transform([prediction])
    return predicted_label[0]

In [18]:
get_labels(204)

'math>>cc-seventh-grade-math>>cc-7th-fractions-decimals'

In [19]:
train_data.iloc[14,1]

'economics-finance-domain>>macroeconomics>>monetary-system-topic'

In [20]:
train_data

,video_transcripts,hierarchy,label
0,In the last couple of videos we saw that we c...,math>>multivariable-calculus>>multivariable-de...,354
1,- What we're going to do in this video is gi...,science>>ap-biology>>natural-selection,422
2,"So once again, we have three equal, or we say...",math>>pre-algebra>>pre-algebra-equations-expre...,384
3,- Liz's math test included a survey question...,math>>engageny-alg-1>>alg1-2,231
4,- The following two equations form a linear s...,math>>algebra-home>>alg-system-of-equations,99
...,...,...,...
4183,- Hello everyone. So this is what I might ca...,math>>multivariable-calculus>>multivariable-de...,354
4184,- Let's try now to subtract some two-digit n...,math>>early-math>>cc-early-math-add-sub-100,226
4185,- Let's say that I have a circle. My best att...,math>>engageny-geo>>geo-5,240
4186,- So let's look at the female reproductive cy...,science>>health-and-medicine>>human-anatomy-an...,497


In [21]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_data["video_transcripts"],test_data["video_transcripts"],train_data["label"],test_data["label"]
val_features,val_labels = val_data["video_transcripts"], val_data["label"]

In [22]:
train_labels.value_counts()

489    99
497    65
505    55
488    54
498    52
       ..
382     1
359     1
195     1
471     1
216     1
Name: label, Length: 569, dtype: int64

In [23]:
test_labels.value_counts()

497    24
489    22
488    17
93     11
485    11
       ..
192     1
191     1
398     1
187     1
291     1
Name: label, Length: 416, dtype: int64

In [24]:
question_answer = train_features.values
categories = train_labels.values

In [25]:
question_answer

array([" In the last couple of videos we saw that we can describe a curves by a position vector-valued function. And in very general terms, it would be the x position as a function of time times the unit vector in the horizontal direction. Plus the y position as a function of time times the unit victor in the vertical direction. And this will essentially describe this-- though, if you can imagine a particle and let's say the parameter t represents time. It'll describe where the particle is at any given time. And if we wanted a particular curve we can say, well, this only applies for some curve-- we're dealing, it's r of t. And it's only applicable between t being greater than a and less than b. And you know, that would describe some curve in two dimensions. Just me just draw it here. This is all a review of really, the last two videos. So this curve, it might look something like that where this is where t is equal to a. That's where t is equal to b. And so r of a will be this vector ri

In [26]:
categories

array([354, 422, 384, ..., 240, 497, 352])

In [27]:
# this method can be used to project from euclidean space to hyperbolic space
def exponential_map(vector):
        norm_v = np.linalg.norm(vector, axis=1)
        coef = np.tanh(norm_v) / norm_v
        second_term = vector * coef[:, None]
        return second_term

In [28]:
# this method can be used to project from euclidean space to hyperbolic space
def tensor_exponential_map(vector):
      vector_norm = vector.norm(dim=-1, p=2, keepdim=True).clamp_min(1e-15)
      gamma_1 = torch.nn.functional.tanh(vector_norm) * (vector / vector_norm)
      return gamma_1

In [29]:
def tensor_log_map(vector):
      vector_norm = vector.norm(dim=-1, p=2, keepdim=True).clamp_min(1e-15)
      gamma_1 = torch.atanh(vector_norm) * (vector / vector_norm)
      return gamma_1

In [30]:

import numpy as np

from bokeh.io import output_file, output_notebook, show
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
from bokeh.models import HoverTool
from bokeh import colors

from gzip import open as gopen

import gensim.models.poincare as poincare
poincare_model = poincare.PoincareModel.load("taxonomy_khan_acad_embedding_20.pkl")
def get_poincare_embeddings_data(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = value.split(">>")
      cleaned_taxonomy.append( list(tok.lower() for tok in value) )
  return cleaned_taxonomy

In [31]:

# course_taxonomy


# course_taxonomy

poincare_emb_data_train = get_poincare_embeddings_data(train_data["hierarchy"].values)
poincare_emb_data_val = get_poincare_embeddings_data(val_data["hierarchy"].values)

In [32]:
poincare_embedding_train =  [exponential_map(np.expand_dims( np.hstack(  [ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] ),axis=0)) for taxonomy in poincare_emb_data_train ]
np.linalg.norm(poincare_embedding_train[1000])

0.899765215781272

In [33]:
poincare_embedding_val=  [exponential_map(np.expand_dims( np.hstack(  [ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] ),axis=0)) for taxonomy in poincare_emb_data_val ]
np.linalg.norm(poincare_embedding_val[100])

0.8857759138716806

In [34]:
max_val_train = 0
max_emb =None
for embedding in poincare_embedding_train:
  val = embedding.shape[1]
  if val >max_val_train:
    max_val_train=val
    max_emb =embedding
max_val_train


60

In [35]:
max_val_val = 0
max_emb =None
for embedding in poincare_embedding_val:
  val = embedding.shape[1]
  if val >max_val_val:
    max_val_val=val
    max_emb =embedding
max_val_val


60

In [36]:
len(set(categories))

569

In [37]:
def get_concat_embedding(poincare_embedding,max_val):
  concatenated_embedding = []
  for embedding in poincare_embedding:
    if embedding.shape[1] < max_val:
      new_embedding = np.append(embedding, np.expand_dims(np.zeros(max_val-embedding.shape[1]),axis=0),axis=1)
    else:
      new_embedding = embedding
    concatenated_embedding.append(np.squeeze(new_embedding,axis=0))
  return concatenated_embedding



In [38]:
concat_embedding_train = get_concat_embedding(poincare_embedding_train,max_val_train)
concat_embedding_val = get_concat_embedding(poincare_embedding_val,max_val_train)

In [39]:
poincare_embeddings_final_train = np.stack(concat_embedding_train, axis=0)
poincare_embeddings_final_train.shape

(4188, 60)

In [40]:
poincare_embeddings_final_val = np.stack(concat_embedding_val, axis=0)
poincare_embeddings_final_val.shape

(924, 60)

In [41]:
num_classes = len(list(set(categories)))
num_classes

569

In [42]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split


In [43]:
import numpy as np
from torch.autograd import Function
class Distance(Function):
    @staticmethod
    def grad(x, v, sqnormx, sqnormv, sqdist, eps):
        alpha = (1 - sqnormx)
        beta = (1 - sqnormv)
        z = 1 + 2 * sqdist / (alpha * beta)
        a = ((sqnormv - 2 * torch.sum(x * v, dim=-1) + 1) / torch.pow(alpha, 2))\
            .unsqueeze(-1).expand_as(x)
        a = a * x - v / alpha.unsqueeze(-1).expand_as(v)
        z = torch.sqrt(torch.pow(z, 2) - 1)
        z = torch.clamp(z * beta, min=eps).unsqueeze(-1)
        return 4 * a / z.expand_as(x)

    @staticmethod
    def forward(ctx, u, v, eps):
        squnorm = torch.clamp(torch.sum(u * u, dim=-1), 0, 1 - eps)
        sqvnorm = torch.clamp(torch.sum(v * v, dim=-1), 0, 1 - eps)
        sqdist = torch.sum(torch.pow(u - v, 2), dim=-1)
        ctx.eps = eps
        ctx.save_for_backward(u, v, squnorm, sqvnorm, sqdist)
        x = sqdist / ((1 - squnorm) * (1 - sqvnorm)) * 2 + 1
        # arcosh
        z = torch.sqrt(torch.pow(x, 2) - 1)
        return torch.log(x + z)

    @staticmethod
    def backward(ctx, g):
        u, v, squnorm, sqvnorm, sqdist = ctx.saved_tensors
        g = g.unsqueeze(-1)
        gu = Distance.grad(u, v, squnorm, sqvnorm, sqdist, ctx.eps)
        gv = Distance.grad(v, u, sqvnorm, squnorm, sqdist, ctx.eps)
        return g.expand_as(gu) * gu, g.expand_as(gv) * gv, None
def distanceTo(vector1,vector2):
        return Distance.apply(vector1,vector2,1e-5)
        # vector1 = vector1.detach().cpu().numpy()
        # vector2 = vector2.detach().cpu().numpy()
        # euclidean_dists = np.linalg.norm(vector1 - vector2)  
        # gamma = 1 + 2 * ((euclidean_dists ** 2) / ((1-(np.linalg.norm(vector1))) * (1-np.linalg.norm(vector2))))  # (1 + neg_size, batch_size)
        # poincare_dists = np.arccosh(gamma) 
        # return torch.tensor(poincare_dists,dtype=torch.float)

In [44]:
import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import geoopt.manifolds.stereographic.math as pm
import geoopt.optim.rsgd as rsgd_
import geoopt.optim.radam as radam_
# from hyrnn.nets import MobiusLinear
from geoopt.tensor import ManifoldParameter
from geoopt.manifolds.stereographic import PoincareBall
from tqdm import tqdm
import geoopt
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import geoopt.optim.rsgd as rsgd_
import geoopt.optim.radam as radam_
# from hyrnn.nets import MobiusLinear
from geoopt.tensor import ManifoldParameter
import time
import argparse
cos = nn.CosineSimilarity(dim=0, eps=1e-6)

def create_ball(ball=None, c=None):
    """
    Helper to create a PoincareBall.
    Sometimes you may want to share a manifold across layers, e.g. you are using scaled PoincareBall.
    In this case you will require same curvature parameters for different layers or end up with nans.
    Parameters
    ----------
    ball : geoopt.PoincareBall
    c : float
    Returns
    -------
    geoopt.PoincareBall
    """
    if ball is None:
        assert c is not None, "curvature of the ball should be explicitly specified"
        ball = geoopt.PoincareBall(c)
    # else trust input
    return ball


class MobiusLinear(torch.nn.Linear):
    def __init__(self, *args, nonlin=None, ball=None, c=1.0, **kwargs):
        super().__init__(*args, **kwargs)
        # for manifolds that have parameters like Poincare Ball
        # we have to attach them to the closure Module.
        # It is hard to implement device allocation for manifolds in other case.
        self.ball = create_ball(ball, c)
        if self.bias is not None:
            self.bias = geoopt.ManifoldParameter(self.bias, manifold=self.ball)
        self.nonlin = nonlin
        self.reset_parameters()

    def forward(self, input):
        return mobius_linear(
            input,
            weight=self.weight,
            bias=self.bias,
            nonlin=self.nonlin,
            ball=self.ball,
        )

    @torch.no_grad()
    def reset_parameters(self):
        torch.nn.init.eye_(self.weight)
        self.weight.add_(torch.rand_like(self.weight).mul_(1e-3))
        if self.bias is not None:
            self.bias.zero_()


# package.nn.functional.py
def mobius_linear(input, weight, bias=None, nonlin=None, *, ball: geoopt.PoincareBall):
    output = ball.mobius_matvec(weight, input)
    if bias is not None:
        output = ball.mobius_add(output, bias)
    if nonlin is not None:
        output = ball.logmap0(output)
        output = nonlin(output)
        output = ball.expmap0(output)
    return output
# Neural Classifierwork
class MulticlassClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(MulticlassClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=False)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = MobiusLinear(768, 384)
        self.fc2 = MobiusLinear(384, 60)

    def forward(self,tokens,masks):
        _, pooled_output,hidden = self.bert(tokens, attention_mask=masks)
        hyerbolic_transform = tensor_exponential_map(hidden[12])
        x = self.fc1(hyerbolic_transform)
        x = self.fc2(x)
        return x

class MyHingeLoss(torch.nn.Module):
    def __init__(self, margin):
        super(MyHingeLoss, self).__init__()
        self.margin = margin


    def forward(self, output, target):
        loss = 0
        for i in range(len(output)):
            text_emb = output[i]
            t_label = target[i]
            j = randint(0, len(output)-1)
            while j == i:
                j = randint(0, len(output)-1)
            t_j = target[j]
            loss += torch.relu( self.margin + \
                            distanceTo(t_label, text_emb) - distanceTo(t_j, text_emb) )
        return loss / len(output)

class MyHingeLoss_cos(torch.nn.Module):

    def __init__(self, margin):
        super(MyHingeLoss_cos, self).__init__()
        self.margin = margin

    def forward(self, output, target):
        loss = 0
        for i in range(len(output)):
            text_emb = output[i]
            t_label = target[i]
            j = randint(0, len(output)-1)
            while j == i:
                j = randint(0, len(output)-1)
            t_j = target[j]
            loss += torch.relu( self.margin - cos(t_label, text_emb) + cos(t_j, text_emb) )
        return loss / len(output)

In [45]:
from transformers import BertModel, AdamW, BertConfig


In [46]:
def dist_without_grad( u, v):
  sqdist = torch.sum((u - v) ** 2, dim=-1)
  squnorm = torch.sum(u ** 2, dim=-1)
  sqvnorm = torch.sum(v ** 2, dim=-1)
  x = 1 + 2 * sqdist / ((1 - squnorm) * (1 - sqvnorm)) + 1e-7
  z = torch.sqrt(x ** 2 - 1)
  return torch.log(x + z)

In [47]:
class FinalClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(FinalClassifier,self).__init__()
        self.model2 = MulticlassClassifier(bert_model_path)
        # self.model2.load_state_dict(torch.load('model_hyperIM_QC/model_weights'))
        self.dropout = nn.Dropout(0.1)
        self.i =0
        for param in self.model2.parameters():
            param.requires_grad = True
        self.mlp = nn.Sequential(
            nn.Linear(896 , 400),
            nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),
            # nn.Linear(mlp_dim, mlp_dim),
            # nn.ReLU(),            
            nn.Linear(400, 572))

    def forward(self,tokens,masks,poincare_tensor):
        output = self.model2.bert(tokens,masks)
        hierarch_output = self.model2(tokens,masks)
        similarity_vector = 1-dist_without_grad(hierarch_output,poincare_tensor.unsqueeze(axis=1))
        # if self.i ==0:
        #   print("similarity_vector",similarity_vector.shape)

        # print("similarity_vector",similarity_vector.shape,hierarch_output.shape,poincare_tensor.shape)
        concat_output = torch.cat((output[1],similarity_vector),dim=1)
        # print("concat_output",concat_output.shape)
        # output_1 = tensor_log_map(output)
        # word_emb = output[2][-1]
        # word_embeddings = 
        concat_out = self.dropout(concat_output)
        x = self.mlp(concat_out)
        return x
model = FinalClassifier('bert-base-uncased')
model.load_state_dict(torch.load('model_hyperIM_khan_acad/model_weights'))
model.cuda()

FinalClassifier(
  (model2): MulticlassClassifier(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
   

In [48]:
from transformers import BertModel, AdamW, BertConfig

# # Loads BertModel, the pretrained BERT model with a single 
# model = MulticlassClassifier('bert-base-uncased')

# # Tell pytorch to run this model on the GPU.
# model.cuda()


In [49]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [50]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [51]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [52]:

test_features = test_features.values
labels = test_labels.values

In [53]:
test_features

array([' -  What I hope to do in this video is get familiar with the notion of an interval, and also think about ways that we can show an interval, or interval notation. Right over here I have a number line. Let\'s say I wanted to talk about the interval on the number line that goes from negative three to two. So I care about this-- Let me use a different color. Let\'s say I care about this interval right over here. I care about all the numbers from negative three to two. So in order to be more precise, I have to be clear. Am I including negative three and two, or am I not including negative three and two, or maybe I\'m just including one of them. So if I\'m including negative three and two, then I would fill them in. So this right over here, I\'m filling negative three and two in, which means that negative three and two are part of this interval. And when you include the endpoints, this is called a closed interval. Closed interval. And I just showed you how I can depict it on a number

In [54]:
labels

array([335, 357,   3, ..., 537, 395, 450])

In [55]:

# course_taxonomy

poincare_emb_data = get_poincare_embeddings_data(test_data["hierarchy"].values)

In [56]:
poincare_embedding =  [exponential_map(np.expand_dims( np.hstack(  [ poincare_model.kv.get_vector(str(x)) for x in taxonomy ] ),axis=0)) for taxonomy in poincare_emb_data ]
np.linalg.norm(poincare_embedding[1000])

0.8837373993769692

In [57]:
max_val = 0
max_emb =None
for embedding in poincare_embedding:
  val = embedding.shape[1]
  if val >max_val:
    max_val=val
    max_emb =embedding
max_val


60

In [58]:
concatenated_embedding = []
for embedding in poincare_embedding:
  if embedding.shape[1] < max_val_train:
    new_embedding = np.append(embedding, np.expand_dims(np.zeros(max_val_train-embedding.shape[1]),axis=0),axis=1)
  else:
    new_embedding = embedding
  concatenated_embedding.append(np.squeeze(new_embedding,axis=0))



In [59]:
poincare_embeddings_final = np.stack(concatenated_embedding, axis=0)
poincare_embeddings_final.shape

(1047, 60)

In [60]:
input_ids = []
attention_masks = []
for sent in test_features:

    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  
test_poincare_tensor = torch.tensor(poincare_embeddings_final,dtype=torch.float)

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, test_poincare_tensor, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# # Create the DataLoader.
# prediction_data = TensorDataset(input_ids, attention_masks, test_poincare_tensor)
# prediction_sampler = SequentialSampler(prediction_data)
# prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [61]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask,b_poincare, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids,b_input_mask,b_poincare)

  logits = outputs

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 1,047 test sentences...


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


    DONE.


print('Positive samples: %d of %d (%.2f%%)' % (final_data.label.sum(), len(final_data.label), (final_data.label.sum() / len(final_data.label) * 100.0)))

In [62]:
pred =  np.argmax(predictions[0],axis=1).flatten()
pred

array([245, 234,  14, 361, 354, 554, 498, 362, 554, 234, 505, 234,  14,
       234, 329, 258, 489,  87, 449, 554, 234, 324, 234, 234, 299,  30,
       485, 324, 449, 449, 485, 505])

In [70]:
final_predictions =[]
flat_predictions = np.concatenate(predictions, axis=0)
for prediction in flat_predictions:
  final_predictions.append(np.argsort(prediction.reshape(1,-1), axis=1)[:,-15:].squeeze())

In [69]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 20)

tmp_rank = tf.nn.top_k(y_pred, 20)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 20) (1047,)
update_recall:  0.5138490926456543
recall 0.5138490926456543
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 538.0, 509.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[398, 367, 337, ..., 110, 102,  81],
       [363, 357, 337, ..., 120, 110,  86],
       [514, 511, 454, ...,   5,   3,   1],
       ...,
       [406, 393, 391, ..., 152,  98,  95],
       [475, 406, 383, ...,  42,  35,  30],
       [557, 530, 476, ..., 157, 149,  12]]), indices=array([[ 1, 17, 13, ...,  9, 10,  3],
       [ 9,  6, 11, ...,  4,  3, 15],
       [ 7, 16,  5, ..., 12,  4, 13],
       ...,
       [13, 16,  4, ...,  1,  6,  5],
       [11, 16,  7, ..., 15, 17, 13],
       [ 1,  7, 16, ...,  6,  3,  9]], dtype=int32))


In [71]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 15)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 15)

tmp_rank = tf.nn.top_k(y_pred, 15)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 15) (1047,)
update_recall:  0.4699140401146132
recall 0.4699140401146132
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 492.0, 555.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[367, 337, 299, ..., 120, 110, 102],
       [363, 357, 337, ..., 174, 121,  86],
       [514, 511, 454, ...,  10,   5,   1],
       ...,
       [406, 393, 370, ..., 171,  98,  95],
       [475, 406, 383, ...,  42,  35,  30],
       [530, 476, 458, ..., 243, 157,  12]]), indices=array([[12,  8, 11, ...,  7,  4,  5],
       [ 4,  1,  6, ...,  7,  0, 10],
       [ 2, 11,  0, ..., 12,  7,  8],
       ...,
       [ 8, 11,  4, ...,  6,  1,  0],
       [ 6, 11,  2, ..., 10, 12,  8],
       [ 2, 11,  3, ...,  5,  1,  4]], dtype=int32))


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 10)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 10)

tmp_rank = tf.nn.top_k(y_pred, 10)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(tmp_rank))

(1047, 10) (1047,)
update_recall:  0.4221585482330468
recall 0.4221585482330468
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 442.0, 605.0, 0.0, 0.0]
TMP_RANK:  TopKV2(values=array([[367, 337, 299, ..., 156, 120, 102],
       [337, 299, 245, ..., 184, 174,  86],
       [511, 346, 329, ...,  10,   5,   1],
       ...,
       [406, 393, 353, ..., 249, 235, 171],
       [475, 406, 235, ...,  42,  35,  30],
       [476, 453, 449, ..., 329, 256, 243]]), indices=array([[7, 3, 6, ..., 4, 2, 0],
       [1, 6, 8, ..., 0, 2, 5],
       [6, 0, 8, ..., 7, 2, 3],
       ...,
       [3, 6, 5, ..., 2, 0, 1],
       [1, 6, 4, ..., 5, 7, 3],
       [6, 3, 9, ..., 8, 4, 0]], dtype=int32))


In [67]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 5)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 5)

tmp_rank = tf.nn.top_k(y_pred, 4)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1047, 5) (1047,)
update_recall:  0.32951289398280803
recall 0.32951289398280803
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 345.0, 702.0, 0.0, 0.0]
TMP_RANK:  [[244 299 367 184 245]
 [ 86 299 222 245 234]
 [152 511  10 329  14]
 ...
 [353 393 311 314 323]
 [ 42 406  35 159 156]
 [419 476 422 329 449]]


In [ ]:
import tensorflow as tf
y_true = np.array(labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 2)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 2)

tmp_rank = tf.nn.top_k(y_pred, 2)
stream_vars = [i for i in tf.local_variables()]

with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1400, 2) (1400,)
update_recall:  0.5078571428571429
recall 0.5078571428571429
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 711.0, 689.0, 0.0, 0.0]
TMP_RANK:  [[417 416]
 [ 56  52]
 [324 322]
 ...
 [225 231]
 [142 143]
 [ 60  10]]


In [ ]:
test_data[test_data["QCLabel"]=="matter_properties of material_REFLECT"]

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold,label
498,Mercury_7132038,7132038,1,A,1,0,Mercury,7,2015,matter_properties of material_REFLECT,What would be the best evidence that two subst...,NaN,Train,Challenge,386
644,Mercury_SC_400861,400861,1,B,1,0,Mercury,5,2015,matter_properties of material_REFLECT,Which object best reflects light? (A) gray doo...,NaN,Train,Easy,386
1102,NYSEDREGENTS_2014_4_13,13,1,B,1,0,NYSEDREGENTS,4,2014,matter_properties of material_REFLECT,A boy can see his face when he looks into a ca...,NaN,Dev,Challenge,386


In [ ]:
final_predictions[644]

array([ 76, 275, 373, 261, 379])

In [ ]:
LE.inverse_transform([161,  93,  41, 400, 378])

array(['Life_functions_environmental conditions', 'FOR_MECH_GENERAL',
       'EARTH_WEATHER_BAROPRESSURE', 'matter_properties of objects_VOL',
       'matter_properties of material_DENSITY'], dtype=object)

In [ ]:
final_predictions[1102]

array([161,  93,  41, 400, 378])

In [ ]:
LE.inverse_transform([161,  93,  41, 400, 378])

array(['Life_functions_environmental conditions', 'FOR_MECH_GENERAL',
       'EARTH_WEATHER_BAROPRESSURE', 'matter_properties of objects_VOL',
       'matter_properties of material_DENSITY'], dtype=object)

In [ ]:
final_predictions[498]

array([298, 338, 333, 329, 328])

In [ ]:
LE.inverse_transform([298, 338, 333, 329, 328])

array(['energy_POTENTIALKINETIC', 'matter_Change of state_MELTING',
       'matter_Change of state_BOILING', 'matter_CHANGES_PHYSICAL',
       'matter_CHANGES_CHEMICAL'], dtype=object)

In [ ]:
labels

tensor([416, 336, 354,  ..., 231, 159,  28])

In [ ]:
final_predictions[0]

array([270, 298,  20, 414, 345, 405, 343, 242, 328, 402, 372, 238, 243,
       412,  66, 346, 401, 407, 417, 416])

In [ ]:
LE.inverse_transform([416])

array(['science_INFERENCE_experiment design'], dtype=object)

In [ ]:
test_data.head()

,questionID,originalQuestionID,totalPossiblePoint,AnswerKey,isMultipleChoiceQuestion,includesDiagram,examName,grade,year,QCLabel,Question,subject,category,fold,label
0,Mercury_409529,409529,1,B,1,0,Mercury,7,2015,science_INFERENCE_experiment design,Robert is a fisherman who wants to find a way ...,NaN,Dev,Challenge,416
1,Mercury_7090790,7090790,1,A,1,0,Mercury,7,2015,matter_Change of state_EVAPoration,Which of these factors causes water to evapora...,NaN,Test,Challenge,336
2,TIMSS_2007_8_pg7,pg7,1,C,1,0,TIMSS,8,2007,matter_chemistry_atomic,Which statement is true about the particles of...,NaN,Train,Challenge,354
3,Mercury_7014455,7014455,1,A,1,0,Mercury,8,2015,energy_LIGHT_GENERICPROP,Which generates waves that are capable of trav...,NaN,Dev,Easy,292
4,NAEP_2000_8_S21+4,S21+4,1,C,1,0,NAEP,8,2000,forces and friction,To keep a heavy box sliding across a carpeted ...,NaN,Test,Challenge,324
